In [1]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [2]:
print(tf.__version__)

2.1.0


In [3]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 16226137595936556607
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 6574135706
locality {
  bus_id: 1
  links {
  }
}
incarnation: 5119259918173827743
physical_device_desc: "device: 0, name: GeForce RTX 2080, pci bus id: 0000:01:00.0, compute capability: 7.5"
]


In [4]:
from PIL import Image

from matplotlib import image
from matplotlib import pyplot as plt
from numpy import asarray
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D

from os import listdir
import os

Using TensorFlow backend.


---
## All Images

In [5]:
# load all images in a list
X = list()
y = list()

for style in listdir('./processed'):
    for filename in listdir('./processed/' + style):
        # load image
        img_data = image.imread('./processed/' + style + '/' + filename)
        # convert pixel values in array to float
        img_data.astype('float32') 
        # store loaded image
        X.append(img_data/255.0) # divide by 255 to standardize values
        
        y.append(style)
        
print(len(X), len(y))

10759 10759


In [6]:
# create a dictionary to contain mapping of values for y
i = 0
style_dict = dict()

for style in np.unique(y):
    style_dict[style] = f'{i}'
    i += 1
    

style_dict

{'Contemporary': '0',
 'Eclectic': '1',
 'Industrial': '2',
 'Minimalistic': '3',
 'Modern': '4',
 'Scandinavian': '5'}

In [7]:
# convert y into a series to apply mapping using dictionary 'style_dict'
y = pd.Series(y)
y = y.map(style_dict)

# convert X and y to array
X = np.array(X)
y = np.array(y)

## train test split

In [8]:
# train test split, train data to be 80% of data
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, shuffle=True, train_size=0.8)

In [9]:
X_train.shape

(8607, 256, 256, 3)

In [10]:
X_test.shape

(2152, 256, 256, 3)

In [11]:
# convert y values into categories
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)

# Modelling

In [12]:
# instantiating model
cnn_model = Sequential()

# Add a convolutional layer.
cnn_model.add(Conv2D(filters = 64,            # number of filters
                     kernel_size = 3,        # height/width of filter
                     activation='relu',      # activation function 
                     input_shape=(256,256,3))) # shape of input (image)
                     
cnn_model.add(MaxPooling2D(pool_size=(2,2))) # dimensions of region of pooling

cnn_model.add(Conv2D(filters = 32,
                     kernel_size=3,
                     activation='relu'))                     
                     
cnn_model.add(MaxPooling2D(pool_size=(2,2)))

cnn_model.add(Conv2D(filters = 16,
                     kernel_size=3,
                     activation='relu'))                     
                     
cnn_model.add(MaxPooling2D(pool_size=(2,2)))
                     
cnn_model.add(Flatten())
                     
# Add a densely-connected layer with 256 neurons.
cnn_model.add(Dense(256, activation='relu'))

# Let's try to avoid overfitting!
cnn_model.add(Dropout(0.1))

# Add a densely-connected layer with 128 neurons.
cnn_model.add(Dense(128, activation='relu'))

# Let's try to avoid overfitting!
cnn_model.add(Dropout(0.1))

# Add a densely-connected layer with 64 neurons.
cnn_model.add(Dense(64, activation='relu'))

# Let's try to avoid overfitting!
cnn_model.add(Dropout(0.1))
                     
# Add a final layer with 6 neurons.
cnn_model.add(Dense(6, activation='softmax'))

- batch size max at 100. insufficient computing power

In [13]:
# Compile model
cnn_model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

# Fit model on training data
history = cnn_model.fit(X_train,
                        y_train,
                        batch_size=50,
                        validation_data=(X_test, y_test),
                        epochs=10,
                        verbose=1)

Train on 8607 samples, validate on 2152 samples
Epoch 1/10
8607/8607 [==============================] - 92s 11ms/step - loss: 1.7427 - accuracy: 0.2261 - val_loss: 1.6963 - val_accuracy: 0.2533
Epoch 2/10
8607/8607 [==============================] - 57s 7ms/step - loss: 1.6985 - accuracy: 0.2515 - val_loss: 1.6996 - val_accuracy: 0.2533
Epoch 3/10
8607/8607 [==============================] - 59s 7ms/step - loss: 1.6567 - accuracy: 0.2757 - val_loss: 1.6903 - val_accuracy: 0.2546
Epoch 4/10
8607/8607 [==============================] - 66s 8ms/step - loss: 1.5858 - accuracy: 0.3225 - val_loss: 1.7293 - val_accuracy: 0.2402
Epoch 5/10
8607/8607 [==============================] - 63s 7ms/step - loss: 1.4223 - accuracy: 0.4158 - val_loss: 1.9457 - val_accuracy: 0.2114
Epoch 6/10
8607/8607 [==============================] - 58s 7ms/step - loss: 1.1807 - accuracy: 0.5365 - val_loss: 2.1614 - val_accuracy: 0.2100
Epoch 7/10
8607/8607 [==============================] - 53s 6ms/step - loss: 0.90